In [1]:
! pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.6 MB/s eta 0:00:00


In [7]:
import json

In [84]:
client = OpenAI(
    base_url="https://api.apiyi.com/v1",
)


In [149]:
def create_func(name, description, parameters):
    return {
        "name": name,
        "description": description,
        "parameters": parameters
    }

def create_parameters(required, properties):
    return {
        "type": "object",
        "properties": properties,
        "required": required
    }

def create_tool(name, description, parameters):
    return {
        "type":"function",
        "function": create_func(name, description, parameters)
    }

tools = []

def append_tools(name, description, parameters):
    tools.append(create_tool(name, description, parameters))

append_tools(
    "analyze_loki_log",
    "从Loki获取日志查询结果",
    create_parameters(
        ["query_str"],
        {"query_str": {"description": "Loki查询字符串，例如：{app=\"grafana\"} |= \"Error\""}}
    )
)

append_tools(
    "modify_config",
    "修改服务配置",
    create_parameters(
        ["service_name", "key", "value"],  # required
        {"service_name": {
            "description": "The name of the service to modify."},
            "key": {"description": "The key to modify."},
            "value": {"description": "The new value for the key."}
        })
    )

append_tools(
    "restart_service",
    "重启指定服务",
    create_parameters(
        ["service_name"],
        {"service_name": {"description": "The name of the service to restart."}}
    )
)

append_tools(
    "apply_manifests",
    "应用清单",
    create_parameters(
        ["resource_type", "images"],
        {"resource_type": {"description": "The type of resource to apply manifests for."},
         "images": {"description": "The images to use for the resource."}}
    )
)

In [150]:
def analyze_loki_log(query_str):
    print("=====================================分析日志=========================")
    print("\n函数调用的参数: ", query_str)
    return json.dumps({"log": "this is error log"})

def modify_config(service_name, key, value):
    print("=====================================修改配置=========================")
    print(f"Modifying config for service: {service_name}")
    print(f"Key: {key}, Value: {value}")
    return "Config modified successfully."


def restart_service(service_name):
    # 假设重启服务需要调用某个API
    print("=====================================重启服务=========================")
    print(f"Restarting service: {service_name}")
    print(f"systemctl restart {service_name}")
    return "Service restarted successfully."

def apply_manifests(resource_type, images):
    print("=====================================应用清单=========================")
    print(f"Applying manifests for {resource_type} with images: {images}")
    print(f"kubectl apply -f manifests/{resource_type}.yaml")
    print("Manifests applied successfully.")
    return "Manifests applied successfully."

In [151]:
##查询需要要调用的函数
def calling(query):
    messages = [
        {
            "role": "system",
            "content": "你是一个 Loki 日志分析助手，你可以帮助用户分析Loki日志，你可以调用多个函数来帮助用户完成任务，并最终尝试分析错误产生的原因",
        },
        {
            "role": "user",
            "content": query,
        },
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # 步骤二：检查 LLM 是否调用了 function
    if tool_calls is None:
        print("not tool_calls")
    if tool_calls:
        available_functions = {
            "analyze_loki_log": analyze_loki_log,
            "modify_config": modify_config,
            "restart_service": restart_service,
            "apply_manifests": apply_manifests,
        }
        messages.append(response_message)
        # 步骤三：把每次 function 调用和返回的信息传给 model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            if function_to_call is None:
              print(f"Function '{function_name}' not found.")
              continue
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        # 步骤四：把 function calling 的结果传给 model，进行对话
        print("=====================================最终结果=========================")
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )
        print(response.choices[0].message.content)


In [176]:
res = calling("第一步：查看app=grafana且关键字包含Error的loki日志，需要分析结果中是否包含'this is error log'；修改redis的配置key=level,value=debug,接着重启redis服务；apply应用清单资源为cluster，镜像为redis:latest")

=====================================分析日志=========================

函数调用的参数:  {app="grafana"} |= "Error"
=====================================修改配置=========================
Modifying config for service: redis
Key: level, Value: debug
=====================================重启服务=========================
Restarting service: redis
systemctl restart redis
=====================================应用清单=========================
Applying manifests for cluster with images: redis:latest
kubectl apply -f manifests/cluster.yaml
Manifests applied successfully.
=====================================最终结果=========================
第一步动作完成了多个操作，具体结果如下：

1. **Loki 日志查询**：
    - 找到了包含`app=grafana`且关键字包含`Error`的日志。
    - 日志内容中确实包含了`this is error log`。

2. **修改 Redis 配置**：
    - 成功修改了Redis配置，将`level`设置为`debug`。

3. **重启 Redis 服务**：
    - Redis 服务已成功重启。

4. **应用清单资源**：
    - 成功应用了清单资源，镜像为`redis:latest`。

如果有其他需要进一步操作的地方，请告诉我！


In [180]:
res = calling("第一步：查看app=grafana且关键字包含Error的loki日志，需要分析结果中是否包含'this is error log'；如果上一步中包含，则修改redis的配置key=level,value=debug,接着重启redis服务；apply应用清单资源为cluster，镜像为redis:latest")

=====================================分析日志=========================

函数调用的参数:  {app="grafana"} |= "Error"
=====================================最终结果=========================
日志中包含关键字 "this is error log"。

继续执行接下来的步骤：

### 修改Redis配置
1. **配置key：** level
2. **配置value：** debug

### 重启Redis服务并应用配置
1. **应用示例资源（cluster）：** 
2. **镜像：** redis:latest

接下来，我将执行以下命令更新Redis配置并重启服务。

#### 更新Redis配置
```sh
redis-cli CONFIG SET level debug
```

#### 重启Redis服务
```sh
kubectl rollout restart deployment/redis-deployment
```

#### 应用清单资源
```sh
kubectl apply -f <path-to-cluster-manifest> --image=redis:latest
```

具体的命令可能需要根据你的环境和部署方式做调整。

是否需要我继续执行这些命令或进一步的帮助？


In [181]:
res = calling("查看 app=grafana 且关键字包含 Error 的日志，如果包含this is error log，则修改loki的配置key=level,value=debug,然后重启redis服务，最后应用清单资源为cluster，镜像为redis:latest")

=====================================分析日志=========================

函数调用的参数:  {app="grafana"} |= "Error"
=====================================最终结果=========================
在查询到的日志中确实包含 "this is error log"。下面我将执行以下操作：
1. 修改Loki的配置，设置`level`键的值为`debug`。
2. 重启Redis服务。
3. 更新应用清单资源为`cluster`，并采用镜像 `redis:latest`。

首先，我将修改Loki的配置。


In [183]:
res = calling("查看 app=grafana 且关键字包含 Error 的日志，如果包含this is error log，则修改loki的配置key=level,value=debug,然后重启redis服务，最后应用清单资源为cluster，镜像为redis:latest")

=====================================分析日志=========================

函数调用的参数:  {app="grafana"} |= "Error"
=====================================最终结果=========================
我们在日志中找到了关键字"this is error log"。接下来，将按顺序执行以下步骤：

1. 修改Loki的配置，设置key为level，value为debug。
2. 重启Redis服务。
3. 部署新的Redis镜像到集群中，镜像为`redis:latest`。

现在我们开始执行这些步骤。

### 修改Loki配置

首先，修改Loki配置，设置key为level，value为debug。

### 重启Redis服务

接下来，重启Redis服务。

### 部署Redis镜像

最后，应用新的清单资源，使用`redis:latest`作为镜像。


In [187]:
calling("帮我修改 gateway 的配置，vendor 修改为 alipay")

=====================================修改配置=========================
Modifying config for service: gateway
Key: vendor, Value: alipay
=====================================最终结果=========================
我已经成功修改了 `gateway` 配置中的 `vendor` 为 `alipay`。请问你需要进一步的帮助吗？例如，确认配置后的状态或者检查相关日志？


In [185]:
calling("帮我重启 gateway 服务")

=====================================重启服务=========================
Restarting service: gateway
systemctl restart gateway
=====================================最终结果=========================
`gateway`服务已经成功重启。现在我们可以开始检查Loki日志，以分析问题的原因。你可以提供更多的上下文信息或特定的日志时间范围吗？


In [186]:
calling("帮我部署一个deployment，镜像是nginx")

=====================================应用清单=========================
Applying manifests for deployment with images: ['nginx']
kubectl apply -f manifests/deployment.yaml
Manifests applied successfully.
=====================================最终结果=========================
部署已经成功完成。Nginx镜像的 Deployment 已经成功创建。如果你需要更多信息或进一步操作，请告诉我。
